Folositi urmatoarele seturi de date:
1. [CPU Computer Hardware](https://archive.ics.uci.edu/ml/datasets/Computer+Hardware); excludeti din dataset coloanele: vendor name, model name, estimated relative performance; se va estima coloana "published relative performance".
1. [Boston Housing](http://archive.ics.uci.edu/ml/machine-learning-databases/housing/)
1. [Wisconsin Breast Cancer](http://www.dcc.fc.up.pt/~ltorgo/Regression/DataSets.html); cautati in panelul din stanga Wisconsin Breast Cancer si urmati pasii din "My personal Notes"
1. [Communities and Crime](http://archive.ics.uci.edu/ml/datasets/communities+and+crime); stergeti primele 5 dimensiuni si trasaturile cu missing values.

In [51]:
import numpy as np
print(f'NumPy version: {np.__version__}')

import pandas as pd
print(f'Pandas version: {pd.__version__}')

import sklearn as sk
from sklearn.impute import SimpleImputer
print(f'Sklearn version: {sk.__version__}')

#NumPy version: 1.19.2
#Pandas version: 1.2.3
#Sklearn version: 0.24.1

NumPy version: 1.19.2
Pandas version: 1.2.3
Sklearn version: 0.24.1


In [39]:
machine: pd.core.frame.DataFrame = pd.read_csv("Data/machine.data", header=None)
x_machine: np.array = machine.iloc[:, 2:8].values
y_machine: np.array = machine.iloc[:, 8].values
y_machine = y_machine.reshape(y_machine.shape[0])

In [40]:
housing: pd.core.frame.DataFrame = pd.read_csv("Data/housing.data", delim_whitespace=True, header=None)
x_housing: np.array = housing.iloc[:, :-1].values
y_housing: np.array = housing.iloc[:, -1].values
y_housing = y_housing.reshape(y_housing.shape[0])

In [41]:
r_wpbc: pd.core.frame.DataFrame = pd.read_csv("Data/r_wpbc.data", header=None)
x_r_wpbc: np.array = r_wpbc.iloc[:, 1:].values
y_r_wpbc: np.array = r_wpbc.iloc[:, 1].values
y_r_wpbc = y_r_wpbc.reshape(y_r_wpbc.shape[0])

In [97]:
communities: pd.core.frame.DataFrame = pd.read_csv("Data/communities.data", header=None)
x_communities: np.array = communities.iloc[:, 1:].drop([3], axis=1).values
y_communities: np.array = communities.iloc[:, 0].values
y_communities = y_communities.reshape(y_communities.shape[0])

# imp = SimpleImputer(missing_values = np.nan, strategy="most_frequent")
# euxoqwxnqwxqw = imp.fit_transform(x_communities)


iowxnnxq=communities.copy()
iowxnnxq=iowxnnxq.drop([3], axis=1).replace('?', np.nan).astype('float64')
print(iowxnnxq)

       0     1        2     4     5     6     7     8     9     10   ...  \
0      8.0   NaN      NaN   1.0  0.19  0.33  0.02  0.90  0.12  0.17  ...   
1     53.0   NaN      NaN   1.0  0.00  0.16  0.12  0.74  0.45  0.07  ...   
2     24.0   NaN      NaN   1.0  0.00  0.42  0.49  0.56  0.17  0.04  ...   
3     34.0   5.0  81440.0   1.0  0.04  0.77  1.00  0.08  0.12  0.10  ...   
4     42.0  95.0   6096.0   1.0  0.01  0.55  0.02  0.95  0.09  0.05  ...   
...    ...   ...      ...   ...   ...   ...   ...   ...   ...   ...  ...   
1989  12.0   NaN      NaN  10.0  0.01  0.40  0.10  0.87  0.12  0.16  ...   
1990   6.0   NaN      NaN  10.0  0.05  0.96  0.46  0.28  0.83  0.32  ...   
1991   9.0   9.0  80070.0  10.0  0.16  0.37  0.25  0.69  0.04  0.25  ...   
1992  25.0  17.0  72600.0  10.0  0.08  0.51  0.06  0.87  0.22  0.10  ...   
1993   6.0   NaN      NaN  10.0  0.20  0.78  0.14  0.46  0.24  0.77  ...   

       118   119   120   121   122   123  124   125   126   127  
0     0.12  0.26  0.2